In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
!pip install transformers datasets evaluate accelerate
!pip3 install torch torchvision torchaudio --upgrade --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import torch
torch.__version__ 

In [ ]:
!python run_glue_no_trainer.py --model_name_or_path bert-base-cased --task_name sst2 --max_length 128 --per_device_train_batch_size 32 --learning_rate 2e-5 --num_train_epochs 3  --output_dir /content/mod_runs/SST2 --seed 0